In [147]:
import numpy as np
import pandas as pd
from copy import deepcopy

In [148]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
import re


In [149]:
# montamos GDrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [150]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DonaldTrumpTweets.csv')

In [151]:
#Compruebo que se han cargado correctamente los datos
len(data)

17216

In [152]:
#ídem
data.head()

,Unnamed: 0,date,id,link,retweet,text,author
0,0,Oct 7,784609194234306560,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump
1,1,Oct 10,785608815962099712,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.c...,DonaldTrump
2,2,Oct 8,784840992734064640,/realDonaldTrump/status/784840992734064641,False,The media and establishment want me out of the...,DonaldTrump
3,3,Oct 8,784767399442653184,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump
4,4,Oct 10,785561269571026944,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\n#MAGA #A...,DonaldTrump


In [153]:
data=data["text"]

In [154]:
data[1]

'Is this really America? Terrible!pic.twitter.com/WiwC61PIFu'

In [155]:
data=data.to_string(index=False)

In [156]:
data[0]

' '

In [157]:
print(data)

   Here is my statement.pic.twitter.com/WAZiGoQqMQ
 Is this really America? Terrible!pic.twitter.c...
 The media and establishment want me out of the...
       Certainly has been an interesting 24 hours!
 Debate polls look great - thank you!\n#MAGA #A...
 WHAT THEY ARE SAYING ABOUT THE CLINTON CAMPAIG...
 Thank you! #MAGA #AmericaFirstpic.twitter.com/...
 I will be in Cincinnati, Ohio tomorrow night a...
 Very little pick-up by the dishonest media of ...
 Thank you Florida- a MOVEMENT that has never b...
 The very foul mouthed Sen. John McCain begged ...
 Disloyal R's are far more difficult than Crook...
 With the exception of cheating Bernie out of t...
 It is so nice that the shackles have been take...
 Our very weak and ineffective leader, Paul Rya...
 Despite winning the second debate in a landsli...
 Wow, @CNN got caught fixing their "focus group...
 .@HillaryClinton is NOT above the law!\n#Debat...
 My team of deplorables will be taking over my ...
 So many self-righteous hypocri

In [158]:
text[30]

'e'

In [159]:
text = data.strip()

print('Corpus length {}'.format(len(data)))

Corpus length 878015


In [160]:
#limpio los caracteres que no me interesan/aportan nada y los links a las imágenes de twitter
text= re.sub('\\n|\xa0|\x92|\u200e|\u200f|pic.twitter.com/\w+', '', text)

In [161]:
# cojo los caracteres únicos que me aparecen en nuestro texto
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

print('Unique chars: {}'.format(len(characters)))

Unique chars: 107


In [162]:
#reviso los caracteres que me aparecen
print(characters) 

[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '£', '«', '®', '´', 'º', 'É', 'è', 'é', '–', '—', '‘', '’', '“', '”', '…', '☆', '☞', 'Ｒ', 'Ｔ']


In [163]:
# pasamos los caracteres del modelo a números

X = []
Y = []
length = len(text)
seq_length = 10
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [164]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [165]:
model = Sequential()
model.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 10, 700)           1965600   
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 700)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 700)               3922800   
_________________________________________________________________
dropout_10 (Dropout)         (None, 700)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 107)               75007     
Total params: 5,963,407
Trainable params: 5,963,407
Non-trainable params: 0
_________________________________________________________________


In [174]:
# Entrenamos la red
history = model.fit(X_modified, Y_modified, validation_split=0.05, batch_size=128, epochs=30, shuffle=True).history

Train on 817268 samples, validate on 43015 samples
Epoch 1/30
817268/817268 [==============================] - 320s 392us/step - loss: 1.4370 - val_loss: 1.7915
Epoch 2/30
817268/817268 [==============================] - 321s 392us/step - loss: 1.4160 - val_loss: 1.7867
Epoch 3/30
817268/817268 [==============================] - 320s 391us/step - loss: 1.4223 - val_loss: 1.7896
Epoch 4/30
817268/817268 [==============================] - 322s 394us/step - loss: 1.4283 - val_loss: 1.7813
Epoch 5/30
817268/817268 [==============================] - 323s 395us/step - loss: 1.4324 - val_loss: 1.7810
Epoch 6/30
817268/817268 [==============================] - 321s 392us/step - loss: 1.4345 - val_loss: 1.7665
Epoch 7/30
817268/817268 [==============================] - 322s 394us/step - loss: 1.4338 - val_loss: 1.7948
Epoch 8/30
817268/817268 [==============================] - 322s 395us/step - loss: 1.4348 - val_loss: 1.7570
Epoch 9/30
817268/817268 [==============================] - 319s 390u

In [181]:
def generate_text(model, string_id):
    string_mapped = deepcopy(string_id)
    full_string = [n_to_char[value] for value in string_mapped]
    
    # Generamos los caracteres
    for i in range(133):
        x = np.reshape(string_mapped,(1,len(string_mapped), 1))
        x = x / float(len(characters))

        pred_index = np.argmax(model.predict(x, verbose=0))
        seq = [n_to_char[value] for value in string_mapped]
        full_string.append(n_to_char[pred_index])

        string_mapped.append(pred_index)
        string_mapped = string_mapped[1:len(string_mapped)]
        
    text = ""
    for char in full_string:
        text = text + char
    return text

In [183]:
#Hacemos una revisión del texto que nos genera
generate_text(model,X[20])

'. Is this the problem is the best thing you can do is deal from my mother the seal poly puertion aao alroier and mosers is the only candidate w'

In [ ]:
#Tras probar varias combinaciones, este es el resultado más acertado que he conseguido